In [3]:
#-*- coding: utf-8 -*-
from google.cloud import bigquery
from datetime import timedelta
from datetime import datetime
import argparse
import time
import sys, os
from time import sleep

In [5]:

# Real배포시 수정사항
# 1. os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"
# 2. parse_parameters(): 안에 막는부분과 푸는 부분


os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"


In [6]:

def log_folder_create(root_path, date_key):
    if not os.path.exists(root_path+'/execute_log/'+batch_yyyymmdd):
        os.makedirs(root_path+'/execute_log/'+batch_yyyymmdd)
        

In [7]:
def sensor_check(query_string):
    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()
    query_result = client.query(query_string, job_config=job_config).result().to_dataframe()['cnt'][0]
    if query_result == 0:
        sensor_onoff = False
    else:
        sensor_onoff = True

    return sensor_onoff


In [8]:
def uf_sensor_loop(step_name, query_string):

    f = open("/home/mi_admin/bigquery_batch_python/execute_log/"+batch_yyyymmdd+"/"+ batch_datetime+"_"+unique_id+"_"+date_key.replace("-", "")+"_"+category+"_"+package_name+"_01.log", 'w')
    f.write("["+step_name+"][start]["+datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+"]\n")
    f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+" [sensor_loop Start]\n")

    sensor_loop = 0
    sensor_onoff = False
    while sensor_onoff == False:
        sensor_onoff = sensor_check(query_string)
        f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + " [Sensor Query : " + query_string + "]\n")
        print('{} [Sensor Query] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),query_string))
        sensor_loop += 1
        if sensor_onoff == False:
            f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + " [Result : Sensor Check is False ("+str(sensor_loop)+")]\n")
        try:
            if sensor_loop > sensor_loop_limit:
                f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + " [sensor_Loop exceeds the sensor_loop_limit(" + str(sensor_loop_limit) + ")]\n")
                f.write("["+step_name+"][fail]["+datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+"]\n")
                f.close()
                print('{} [sensor_Loop exceeds the sensor_loop_limit({})]'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"), sensor_loop_limit))
                print('{} [Result : Sensor Check is Fail] \n'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))
                f.close()
                return 1
                #raise ValueError('sensor_Loop exceeds the sensor_loop_limit({})'.format(sensor_loop_limit))
        except ValueError:
            f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + " [exception Error]\n")
            f.write("["+step_name+"][fail]["+datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+"]\n")
            f.close()
            raise
        sleep(10)

    f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + " [Result : Sensor Check is True (" + str(sensor_loop) + ")]\n")
    f.write("["+step_name+"][end]["+datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+"]\n")
    print('{} [Result : Sensor Check is True ({})] \n'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),sensor_loop))
    f.close()
    return 0


In [39]:
def main(meta_tuple):
    global date_key
    global batch_datetime
    global unique_id

    # Input Parameter
    date_key = meta_tuple[0]
    batch_datetime = meta_tuple[1]
    unique_id = meta_tuple[2]


    # Config
    global sensor_loop_limit
    sensor_loop_limit = 360*10    #10초단위
    sensor_loop = 0

    # Information Data
    global root_path
    global category
    global package_name
    root_path = "/home/mi_admin/bigquery_batch_python"
    category = "sensor"
    package_name = 'tier0_bi_globalad_log_sensor'
    last_rows = 0

    # Variable
    global batch_yyyymmdd
    batch_yyyymmdd = batch_datetime[0:8]

    #Log Folder Create
    log_folder_create(root_path, batch_yyyymmdd)

    ###############################################################################################
    # 현재일자 = 작업일자 + 1 (자동배치)인 경우에는 Skip
    # 현재시각이 12시 이전이면, 전일자 로그의 16~17시 데이터가 존재하는지?
    # 현재시각이 12시 이후이면, 당일자 로그테이블이 존재하는지?
    step_name = 'uf_sensor_loop'
    if datetime.now().strftime("%Y-%m-%d") == (datetime.strptime(date_key,'%Y-%m-%d')+timedelta(days=1)).strftime('%Y-%m-%d') and int(datetime.now().strftime("%H")) < 12 :
        query_string = "Select Count(*) as cnt From Tier0_BI.Log_GlobalAd_"+date_key.replace("-", "")+"""
                                            Where regdatetime >= '"""+date_key+" 16:00:00'"+"""
                                              And regdatetime <  '"""+date_key+" 17:00:00'"
        print('{} [Sensor Check (AM)] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),query_string))
        if uf_sensor_loop(step_name, query_string) == 0:
            return '(success)', package_name, last_rows
        else:
            return '(limit)', package_name, last_rows
    elif datetime.now().strftime("%Y-%m-%d") == (datetime.strptime(date_key,'%Y-%m-%d')+timedelta(days=1)).strftime('%Y-%m-%d') and int(datetime.now().strftime("%H")) >=12:
        query_string = "SELECT Count(*) as cnt FROM Tier0_BI.__TABLES__ WHERE table_id = 'Log_GlobalAd_"+datetime.now().strftime("%Y%m%d")+"'"
        print('{} [Sensor Check (PM)] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),query_string))
        if uf_sensor_loop(step_name, query_string) == 0:
            return '(success)', package_name, last_rows
        else:
            return '(limit)', package_name, last_rows
    elif datetime.now().strftime("%Y-%m-%d") < (datetime.strptime(date_key,'%Y-%m-%d')+timedelta(days=1)).strftime('%Y-%m-%d'):
        print('{} [Sensor Check (Future Day)] Stop'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))
        return '(future)', package_name, last_rows
    else:
        query_string = "Select 1 as cnt"
        print('{} [Sensor Check (Old Day)] Pass'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))
        return '(old)', package_name, last_rows

    ###############################################################################################
    # return : (success)/(old)/(limit)/(future)
    

In [40]:

def parse_parameters():
    #     print('\nprocessing parse_parameters() ...')

    #################################################
    # 개발 완료되면 풀어야 함. !!!!!!!!!!!!!!!!!!!
    #################################################
    parser = argparse.ArgumentParser(description='시스템 구동에 필요한 파라미터를 입력하세요.')
    parser.add_argument("--date_key", help="배치작업 기준일자를 YYYY-MM-DD 형태로 입력하세요. 반드시 입력해야 하는 값입니다.", required=True)
    parser.add_argument("--batch_datetime", help="배치실행일자를 YYYYMMDD_HH24MISS 형태로 입력하세요. 반드시 입력해야 하는 값입니다.", required=True)
    parser.add_argument("--unique_id", help="배치 Unique_id를 입력하세요. 반드시 입력해야 하는 값입니다.", required=True)
    args = parser.parse_args()

    date_key = args.date_key
    batch_datetime = args.batch_datetime
    unique_id = args.unique_id

    #################################################
    # 개발 완료되면 막아야 함. !!!!!!!!!!!!!!!!!!!
    #date_key = '2018-05-10'
    #batch_datetime = '20180531123456'
    #unique_id = 'unique_id'
    #################################################


    try:
        if len(date_key.split('-')) != 3:
            raise ValueError('\'date\' parameter seems wrong. It must be in \'YYYY-MM-DD\' format.')
    except ValueError:
        raise

    print('[Start] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))        
    print('- Input parameter : --date_key = {}'.format(date_key))
    print('- Input parameter : --batch_datetime = {}'.format(batch_datetime))
    print('- Input parameter : --unique_id = {}'.format(unique_id))


    return date_key, batch_datetime, unique_id


In [41]:
if __name__ == '__main__':
    start_time = time.time()
    result = 0
    # python3 /home/younseun/bigquery_batch_python/bigquery_batch_python/source/sensor/tier0_bi_globalad_log_sensor.py --date_key 2018-06-13 --batch_datetime 20180614_123456 --unique_id abcdefghij
    #result, package_name, last_rows, exec_log_filename = main(['2018-06-16','20180621_163403','e12f9b7204'])
    result, package_name, last_rows = main(parse_parameters())
    print('[Complete] {} {} {} total elapsed : {} seconds [lastrows] {} '.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"), str(result), package_name, time.time()-start_time, str(last_rows)))

    # return 0 : success / old
    # return 1 : limit     
    # return 2 : future
    return_value = 0
    if result == '(success)' or result == '(old)':
        return_value = 0
    else:
        return_value = 1
            
    sys.exit(return_value)



2018-06-13 14:51:24.045685 [Sensor Check (Old Day)] Pass
2018-06-13 14:51:27.363404 [Result : Sensor Check is True (1).] 

[Complete] tier0_bi_globalad_log_sensor total elapsed : 3.318885087966919 seconds
